In [51]:
import numpy as np
import pandas as pd
#import torch
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
from torch.utils.data import Dataset, DataLoader

ModuleNotFoundError: No module named 'torch'

In [44]:
def get_specgram(dir_to_GAMEEMO, patient, game):
    # Reading from the csv data set (can do matlab as well) using pandas. 

    #Patient = 'S01' #The Patient
    #Game = 'G1' #The game

    #You can also just paste in the Directory of the csv file - on windows you may have to change the slash direction
    DirComb = f'{dir_to_GAMEEMO}/({patient})/Preprocessed EEG Data/.csv format/{patient}{game}AllChannels.csv'

    df=pd.read_csv(DirComb, sep=',',header=None)
    d = np.array(df) #Switching from pandas to numpy array as this might be more comfortable for people
    d = np.delete(d,0,0) # Deleting header
    d = np.delete(d,-1,1).astype(float) #and erroneous last column
    
    for idx, d2 in enumerate(d.T):
        _, _, Sxx = spectrogram(d2,fs=120)
        if idx == 0:
            full_spec = Sxx
        else:
            full_spec = np.append(full_spec, Sxx, axis=1)
    return full_spec

In [45]:
basedir = 'C:/Users/katja/OneDrive/Dokumente/PhD/Courses/Neuromatch/'
labeldir = f'{basedir}GameLabels.csv' #Path to GAMEEMO
gamedir = f'{basedir}archive/GAMEEMO/'

get_specgram(gamedir, 'S01', 'G1')

array([[2.82326543e+00, 3.65090824e-03, 4.92888047e-01, ...,
        6.89887282e-01, 5.05342218e-01, 1.72666483e-01],
       [9.07557623e+00, 2.13118976e+00, 1.27050027e+01, ...,
        2.54881363e+00, 7.12427803e-01, 4.28823487e+00],
       [1.52908893e+01, 7.47031177e-01, 3.39653787e+00, ...,
        1.26804526e+00, 6.55939408e-01, 1.85234790e+00],
       ...,
       [1.60615583e-01, 1.59936037e-03, 7.95917541e-03, ...,
        2.19242904e-03, 4.84510706e-02, 1.22700911e-02],
       [1.23125458e-01, 1.27450902e-02, 2.81848747e-02, ...,
        1.43611098e-01, 8.30239996e-02, 5.38889235e-02],
       [3.26482462e-02, 4.49888105e-02, 9.20906132e-03, ...,
        1.38731491e-02, 1.61095319e-03, 5.20108067e-04]])

In [52]:
labels_df = pd.read_csv(labeldir)


labels_df['full_specgram_1'] = [[]] * len(labels)
for idx in range(len(labels)):
    game = f'G{int(idx % 4)+1}'
    subject = f'S{int(idx/4)+1:02d}'
    
    labels_df['full_specgram_1'][idx] = get_specgram(gamedir, subject, game)

print(labels_df.iloc[0, :])

C:\Users\katja\anaconda3\envs\suite2p\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


subject                                                            1
game                                                               1
gender                                                             F
age                                                               24
disturbance                                                        N
experience                                                         N
memory                                                             N
satisfied                                                          2
boring                                                             8
horrible                                                           1
calm                                                               3
funny                                                              2
valence                                                            2
arousal                                                            3
full_specgram_1    [[2.82326542676

In [50]:
class CerealTimeKillersDataset(Dataset):
    """Spectrogram dataset."""

    def __init__(self, df):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ori_dataframe = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        spectrogram = self.ori_dataframe.iloc[idx, 14]
        labels = self.landmarks_frame.iloc[idx, :14]
        labels = np.array([labels])
        sample = {'spectrogram': spectrogram, 'labels': labels}

        return sample

NameError: name 'Dataset' is not defined

In [ ]:
final_dataset = CerealTimeKillersDataset(labels_df)
dataloader = DataLoader(final_dataset, batch_size=4, shuffle=True,
                        num_workers=0)